In [12]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [13]:
from azureml.core import Workspace

# Parameters (make sure these are correct)
subscr_id = "" 
resc_name = ""
worksp_name = ""

# Get a handle to the workspace
ml_client = MLClient(
    credential = credential,
    subscription_id = subscr_id,
    resource_group_name = resc_name,
    workspace_name = worksp_name,
)

# ws 
ws = Workspace(subscr_id, resc_name, worksp_name)

In [15]:
from azure.ai.ml.entities import AmlCompute

compute_target = "august-test-compute"

# let's see if the compute target exists
cluster = ml_client.compute.get(compute_target)
print(
    f"You have a cluster named {compute_target}, we'll use it as is."
)

# If you get the following error:
# 'ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials'
# You may need to authenticate the compute (somewhere above)

# If you want to create a compute from Python:

# try:
#     # let's see if the compute target already exists
#     cluster = ml_client.compute.get(compute_target)
#     print(
#         f"You already have a cluster named {compute_target}, we'll reuse it as is."
#     )

# except Exception:
#     print("Creating a new gpu compute target...")

    # # Let's create the Azure ML compute object with the intended parameters
    # cluster = AmlCompute(
    #     # Name assigned to the compute cluster
    #     name="cpu-cluster",
    #     # Azure ML Compute is the on-demand VM service
    #     type="amlcompute",
    #     # VM Family
    #     size="Standard_E4ds_v4",
    #     # Minimum running nodes when there is no job running
    #     min_instances=0,
    #     # Nodes in cluster
    #     max_instances=4,
    #     # How many seconds will the node running after the job termination
    #     idle_time_before_scale_down=180,
    #     # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
    #     tier="Dedicated",
    # )

    # # Now, we pass the object to MLClient's create_or_update method
    # cluster = ml_client.begin_create_or_update(cluster).result()

print(
    f"AMLCompute with variable name 'compute_target', the compute size is {cluster.size}"
)

You have a cluster named august-test-compute, we'll use it as is.
AMLCompute with variable name 'compute_target', the compute size is STANDARD_DS11_V2


In [16]:
# For job
env_name = "test4-environment@latest"

# Or
from azureml.core import Environment
mycustomenv = Environment(name="test4-environment@latest")

In [17]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential
from azureml.core import Workspace, Model

# Load registered model
model_asset = Model(ws, name='test-model') # latest version, otherwise ', version = ...'

# Download to local folder
model_asset.download(exist_ok=True) 

'model'

In [18]:
from azure.ai.ml import command
from azure.ai.ml import Input
import os

# Load data asset containing raster(s) for prediction
test_data_asset = ml_client.data.get("test-test-dataset", version="1")

# Please name here the rasters in the dataset for prediction (check if correct name)
rasters = ['steenplaat_vlucht1', 'waterdunen_vlucht2']
create_subset = "yes" # ('yes' or 'no') subset takes less time



In [ ]:
# Set job and run
job = command(
    code=".", 
    command='python detectron2_predict.py --rasterdata ${{inputs.rasterdata}} --rasternames${{rasters}} --create_subset${{create_subset}}',
    inputs={
        "rasterdata": Input(path=test_data_asset.id,
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RO_MOUNT # Mount (.RO_MOUNT) or download (.DOWNLOAD)
        )
    },
    environment=env_name,
    outputs={},  # Add any required outputs here
 )

ml_client.jobs.create_or_update(job)